In [328]:
import os
import sys
import time
import sqlite3
import pandas as pd
import yfinance as yf
import logging
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

RETRY_ATTEMPTS = 3
RETRY_DELAY = 1  
REQUEST_DELAY = 0.5  
DB_NAME = "etf_data.db"  
LOG_FILE = "etf_pipeline.log"

# lead in log
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))  

def download_data(tickers: list[str]) -> tuple[pd.DataFrame, list[str]]:
    """Download historical data for a list of ticker symbols."""
    all_data = []
    failed_tickers = []
    
    # time arranging:1 yr
    end_date = datetime.today()
    start_date = end_date - timedelta(days=365)
    
    logging.info(f"Starting download for {len(tickers)} tickers")
    logging.info(f"Current working directory: {os.getcwd()}")
    
    for i, ticker in enumerate(tickers):
        clean_ticker = ticker.replace('.', '-').strip()
        if not clean_ticker:
            logging.warning(f"Skipping empty ticker at position {i}")
            continue
            
        #mark down starting time
        start_time = time.time()
        
        for attempt in range(RETRY_ATTEMPTS):
            try:
                #retrive ETF data from yfinance
                logging.debug(f"Downloading data for {clean_ticker} (attempt {attempt+1})")
                etf = yf.Ticker(clean_ticker)
                
                #tesing yfinance index
                logging.debug(f"Ticker info keys: {list(etf.info.keys())}")
                
                hist = etf.history(
                    start=start_date,
                    end=end_date,
                    interval='1d'
                )
                
                #testing downloaded data
                if hist is None:
                    logging.warning(f"Hist is None for {clean_ticker}")
                    continue
                
                logging.debug(f"Downloaded data shape: {hist.shape}")
                if hist.empty:
                    logging.warning(f"No data for {clean_ticker} (attempt {attempt+1})")
                    time.sleep(RETRY_DELAY)
                    continue
                    
                # setting data format
                hist.reset_index(inplace=True)
                hist['Ticker'] = clean_ticker
                hist.rename(columns={
                    'Date': 'date',
                    'Open': 'open',
                    'High': 'high',
                    'Low': 'low',
                    'Close': 'close',
                    'Volume': 'volume'
                }, inplace=True)
                
                #column selection
                hist = hist[['date', 'Ticker', 'open', 'high', 'low', 'close', 'volume']]
                
                # adding successfully fetched data
                all_data.append(hist)
                elapsed = time.time() - start_time
                logging.info(f"[{i+1}/{len(tickers)}] Downloaded {len(hist)} days for {clean_ticker} in {elapsed:.2f}s")
                
                #savng into .csv
                hist.to_csv(f"debug_{clean_ticker}.csv", index=False)
                logging.debug(f"Saved debug CSV for {clean_ticker}")
                
                break  
                
            except Exception as e:
                logging.error(f"Attempt {attempt+1} failed for {ticker}: {str(e)}", exc_info=True)
                time.sleep(RETRY_DELAY)
        else:
            # all failed
            failed_tickers.append(clean_ticker)
            logging.error(f"Failed to download {clean_ticker} after {RETRY_ATTEMPTS} attempts")
        
        # adding download interval
        time.sleep(REQUEST_DELAY)
    
    # merge all the database
    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        logging.info(f"Successfully downloaded data for {len(all_data)} ETFs, {len(failed_tickers)} failed")
        
        # saving merged file-for after analysis
        result_df.to_csv("all_etf_data_debug.csv", index=False)
        logging.info("Saved debug CSV with all ETF data")
        
        return result_df, failed_tickers
    else:
        logging.error("No data downloaded for any ETF")
        return pd.DataFrame(), failed_tickers

def save_to_database(df: pd.DataFrame, table_name: str) -> bool:
    if df.empty:
        logging.warning(f"Not saving empty DataFrame to {table_name}")
        return False
        
    try:
        # select saving path
        db_path = os.path.abspath(DB_NAME)
        logging.info(f"Saving to database: {db_path}")
        
        # check wether the file is readable
        if os.path.exists(db_path):
            logging.info(f"Database file exists, size: {os.path.getsize(db_path)} bytes")
        else:
            logging.info("Creating new database file")
        
        with sqlite3.connect(DB_NAME) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = cursor.fetchall()
            logging.info(f"Existing tables: {tables}")
            
            # saving methods
            if table_name == 'etf_info':
                # ETF database-replace
                df.to_sql(table_name, conn, if_exists='replace', index=False)
            else:
                # pricedata-append
                df.to_sql(table_name, conn, if_exists='append', index=False)
                
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables_after = cursor.fetchall()
            logging.info(f"Tables after operation: {tables_after}")
            
            # checking row number
            cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
            row_count = cursor.fetchone()[0]
            logging.info(f"Table '{table_name}' now has {row_count} rows")
            
        logging.info(f"Saved {len(df)} records to {table_name}")
        
        # 保存CSV备份
        csv_path = f"{table_name}.csv"
        df.to_csv(csv_path, index=False)
        logging.info(f"Saved CSV backup to {csv_path}")
        
        return True
    except sqlite3.OperationalError as e:
        logging.error(f"SQLite operational error: {str(e)}")
        return False
    except Exception as e:
        logging.error(f"Database operation failed for {table_name}: {str(e)}", exc_info=True)
        return False

def retrieve_etf_tickers() -> pd.DataFrame:
    logging.warning("Using hardcoded ETF list for testing")
    
    # sample ticker testing
    test_tickers = [
        "SPY",  # SPDR S&P 500 ETF
        "QQQ",  # Invesco QQQ Trust
        "DIA",  # Dow Jones Industrial Average ETF
        "IWM",  # iShares Russell 2000 ETF
        "GLD",  # SPDR Gold Shares
        "VTI",  # Vanguard Total Stock Market ETF
        "VOO",  # Vanguard S&P 500 ETF
        "IVV",  # iShares Core S&P 500 ETF
        "VEA",  # Vanguard FTSE Developed Markets ETF
        "VWO",  # Vanguard FTSE Emerging Markets ETF
    ]
    
    # create a stimulation data base
    etf_data = {
        'symbol': test_tickers,
        'name': [f"Test ETF {i}" for i in range(len(test_tickers))],
        'sector': ["Diversified"] * len(test_tickers),
        'assetManager': ["Test Manager"] * len(test_tickers),
        'marketCap': [1000000000] * len(test_tickers)
    }
    
    return pd.DataFrame(etf_data)

def main():
    # 1. fetching etf
    etf_tickers = retrieve_etf_tickers()
    logging.info(f"Retrieved {len(etf_tickers)} ETF tickers")
    
    # 2. download price 
    ticker_list = etf_tickers['symbol'].tolist()
    price_data, failed_tickers = download_data(ticker_list)
    
    # 3.save merged files
    if not price_data.empty:
        save_success = save_to_database(price_data, "etf_prices")
        if save_success:
            logging.info("Price data saved to database successfully")
        else:
            logging.error("Failed to save price data to database")
    else:
        logging.error("No price data to save")
    
    # 4. mark down failed value
    if failed_tickers:
        failed_df = pd.DataFrame(failed_tickers, columns=['ticker'])
        save_to_database(failed_df, "failed_downloads")
        logging.info(f"Saved {len(failed_tickers)} failed tickers")
    
    # 5. saving etf
    save_to_database(etf_tickers, "etf_info")
    
    logging.info("Pipeline completed")

    # setting  a detailed log for adjust
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.DEBUG)
    logging.info(f"Current working directory: {os.getcwd()}")
    
    # main procedure 
    main()
    
    # double check
    if os.path.exists(DB_NAME):
        logging.info(f"Database created successfully at {os.path.abspath(DB_NAME)}")
        logging.info(f"Database size: {os.path.getsize(DB_NAME)} bytes")
    else:
        logging.error("Database file not created")

Current working directory: /Users/rayxu


Current working directory: /Users/rayxu
Current working directory: /Users/rayxu
Current working directory: /Users/rayxu


Current working directory: /Users/rayxu


Current working directory: /Users/rayxu


Current working directory: /Users/rayxu
Using hardcoded ETF list for testing


...

Database created successfully at /Users/rayxu/etf_data.db
Database size: 1257472 bytes


Database size: 1257472 bytes
Database size: 1257472 bytes
Database size: 1257472 bytes


Database size: 1257472 bytes


Database size: 1257472 bytes


Database size: 1257472 bytes


In [312]:
import requests
import pandas as pd
import sqlite3
import yfinance as yf
import time
import logging
from datetime import datetime, timedelta

# Configuration - consider moving to a config file
FMP_API_KEY = "WkeiZXdoyVCcrF6JAMg0dfpdNAnIBQgC" 
TICKER_SOURCE_URL = f"https://financialmodelingprep.com/api/v3/etf/list?apikey={FMP_API_KEY}"
DB_NAME = "etf_data.db"
HISTORY_PERIOD = "1y"
RETRY_ATTEMPTS = 3
RETRY_DELAY = 2  # seconds
REQUEST_DELAY = 0.5  # seconds to avoid rate limits
MAX_TICKERS = 50  # Limit for testing (remove in production)

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('etf_pipeline.log'),
        logging.StreamHandler()
    ]
)

def retrieve_etf_tickers() -> pd.DataFrame:
    """
    Retrieves live US ETF tickers from Financial Modeling Prep API
    Returns DataFrame with columns: [symbol, name, marketCap, sector, assetManager]
    """
    try:
        logging.info(f"Retrieving ETF tickers from {TICKER_SOURCE_URL}")
        response = requests.get(TICKER_SOURCE_URL, timeout=15)
        response.raise_for_status()
        etf_data = response.json()
        
        if not etf_data:
            logging.error("No ETF data returned from API")
            return pd.DataFrame()
            
        # Extract relevant information
        etfs = []
        for etf in etf_data:
            etfs.append({
                'symbol': etf.get('symbol', ''),
                'name': etf.get('name', ''),
                'sector': etf.get('sector', 'N/A'),
                'assetManager': etf.get('assetManager', 'N/A'),
                'marketCap': etf.get('marketCap', 0)
            })
        
        df = pd.DataFrame(etfs)
        logging.info(f"Retrieved {len(df)} ETFs")
        return df
    
    except requests.exceptions.RequestException as e:
        logging.error(f"API request failed: {str(e)}")
        return pd.DataFrame()
    except Exception as e:
        logging.error(f"Unexpected error in retrieve_etf_tickers: {str(e)}")
        return pd.DataFrame()

def download_etf_history(tickers: list) -> (pd.DataFrame, list):
    """
    Downloads historical price data for a list of ETF tickers
    Returns tuple: (combined DataFrame of historical data, list of failed tickers)
    """
    all_data = []
    failed_tickers = []
    
    # Apply safety limit during testing
    if MAX_TICKERS > 0:
        tickers = tickers[:MAX_TICKERS]
        logging.info(f"Limiting to first {MAX_TICKERS} tickers for testing")
    
    logging.info(f"Starting historical download for {len(tickers)} ETFs")
    
    for i, ticker in enumerate(tickers):
        # Skip empty tickers
        if not ticker or not isinstance(ticker, str):
            logging.warning(f"Skipping invalid ticker: {ticker}")
            continue
            
        for attempt in range(RETRY_ATTEMPTS):
            try:
                # Clean ticker symbol
                clean_ticker = ticker.strip().upper()
                
                etf = yf.Ticker(clean_ticker)
                hist = etf.history(period=HISTORY_PERIOD, interval='1d')
                
                if hist.empty:
                    logging.warning(f"No data for {clean_ticker} (attempt {attempt+1})")
                    time.sleep(RETRY_DELAY)
                    continue
                    
                # Prepare data for storage
                hist.reset_index(inplace=True)
                hist['Ticker'] = clean_ticker
                hist.rename(columns={
                    'Date': 'date',
                    'Open': 'open',
                    'High': 'high',
                    'Low': 'low',
                    'Close': 'close',
                    'Volume': 'volume'
                }, inplace=True)
                
                # Select only relevant columns
                hist = hist[['date', 'Ticker', 'open', 'high', 'low', 'close', 'volume']]
                
                all_data.append(hist)
                logging.info(f"[{i+1}/{len(tickers)}] Downloaded {len(hist)} days for {clean_ticker}")
                break
                
            except Exception as e:
                logging.warning(f"Attempt {attempt+1} failed for {ticker}: {str(e)}")
                time.sleep(RETRY_DELAY)
        else:
            failed_tickers.append(ticker)
            logging.error(f"Failed to download {ticker} after {RETRY_ATTEMPTS} attempts")
        
        # Add delay between requests to avoid rate limiting
        time.sleep(REQUEST_DELAY)
    
    # Combine all successful downloads
    if all_data:
        return pd.concat(all_data, ignore_index=True), failed_tickers
    else:
        logging.warning("No historical data downloaded")
        return pd.DataFrame(), failed_tickers

def save_to_database(df: pd.DataFrame, table_name: str) -> bool:
    """
    Saves DataFrame to SQLite database
    Returns True on success, False on failure
    """
    if df.empty:
        logging.warning(f"Not saving empty DataFrame to {table_name}")
        return False
        
    try:
        with sqlite3.connect(DB_NAME) as conn:
            # Use if_exists='replace' for metadata, 'append' for price data
            if_exists = 'replace' if table_name == 'etf_info' else 'append'
            df.to_sql(table_name, conn, if_exists=if_exists, index=False)
        logging.info(f"Saved {len(df)} records to {table_name}")
        return True
    except Exception as e:
        logging.error(f"Database operation failed for {table_name}: {str(e)}")
        return False

def main():
    """
    Main orchestration function for the ETF pipeline
    """
    logging.info("Starting ETF data pipeline")
    start_time = time.time()
    
    # Step 1: Retrieve current ETF metadata
    etf_df = retrieve_etf_tickers()
    
    if etf_df.empty:
        logging.error("Failed to retrieve ETF information. Exiting.")
        return
    
    # Step 2: Save metadata to database
    save_to_database(etf_df, 'etf_info')
    
    # Step 3: Download historical prices
    tickers = etf_df['symbol'].tolist()
    historical_data, failed_tickers = download_etf_history(tickers)
    
    # Step 4: Save historical data to database
    if not historical_data.empty:
        save_to_database(historical_data, 'historical_prices')
    
    # Step 5: Report results
    duration = time.time() - start_time
    success_count = len(tickers) - len(failed_tickers)
    
    logging.info(f"Pipeline completed in {duration:.2f} seconds")
    logging.info(f"Successfully processed {success_count}/{len(tickers)} ETFs")
    
    if failed_tickers:
        logging.warning(f"Failed tickers: {', '.join(failed_tickers)}")
    
    # Print final status
    print("\n=== ETF Pipeline Summary ===")
    print(f"Retrieved {len(etf_df)} ETF records")
    print(f"Downloaded history for {success_count} ETFs")
    print(f"Failed downloads: {len(failed_tickers)}")
    print(f"Database: {DB_NAME}")
    print(f"Total time: {duration:.2f} seconds")

if __name__ == "__main__":
    main()

Starting ETF data pipeline


Starting ETF data pipeline
Starting ETF data pipeline
Starting ETF data pipeline


Starting ETF data pipeline


Starting ETF data pipeline


Retrieving ETF tickers from https://financialmodelingprep.com/api/v3/etf/list?apikey=WkeiZXdoyVCcrF6JAMg0dfpdNAnIBQgC


...

Failed tickers: FFTI, 3121.HK, HFXE, DFU.TO, UCOM, BRGL, XFS.TO, UBJ.AX


Failed tickers: FFTI, 3121.HK, HFXE, DFU.TO, UCOM, BRGL, XFS.TO, UBJ.AX
Failed tickers: FFTI, 3121.HK, HFXE, DFU.TO, UCOM, BRGL, XFS.TO, UBJ.AX
Failed tickers: FFTI, 3121.HK, HFXE, DFU.TO, UCOM, BRGL, XFS.TO, UBJ.AX


Failed tickers: FFTI, 3121.HK, HFXE, DFU.TO, UCOM, BRGL, XFS.TO, UBJ.AX


Failed tickers: FFTI, 3121.HK, HFXE, DFU.TO, UCOM, BRGL, XFS.TO, UBJ.AX



=== ETF Pipeline Summary ===
Retrieved 13034 ETF records
Downloaded history for 13026 ETFs
Failed downloads: 8
Database: etf_data.db
Total time: 82.18 seconds


In [326]:
!pip install reportlab
from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

class ETFReportGenerator:
    def __init__(self, validation_dir="validation_results"):
        self.validation_dir = validation_dir
        self.report_dir = os.path.join(validation_dir, "reports")
        self.visualizations_dir = os.path.join(validation_dir, "visualizations")
        
        # Create report directory
        os.makedirs(self.report_dir, exist_ok=True)
        os.makedirs(self.visualizations_dir, exist_ok=True)
        
        # Report styles
        self.styles = getSampleStyleSheet()
        self.title_style = ParagraphStyle(
            'Title',
            parent=self.styles['Heading1'],
            fontSize=18,
            alignment=1,
            spaceAfter=12,
            textColor=colors.HexColor("#2c3e50")
        )
        self.section_style = ParagraphStyle(
            'Section',
            parent=self.styles['Heading2'],
            fontSize=14,
            spaceBefore=12,
            spaceAfter=6,
            textColor=colors.HexColor("#3498db")
        )
        self.normal_style = self.styles['Normal']
        self.table_style = TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#2c3e50")),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
            ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor("#ecf0f1")),
            ('GRID', (0, 0), (-1, -1), 1, colors.HexColor("#bdc3c7")),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
        ])
    
    def generate_full_report(self, price_data=None):
        """Generate full ETF data quality report"""
        # Generate timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        report_filename = os.path.join(self.report_dir, f"ETF_Data_Quality_Report_{timestamp}.pdf")
        
        # Create PDF document
        doc = SimpleDocTemplate(
            report_filename,
            pagesize=landscape(letter),
            rightMargin=36,
            leftMargin=36,
            topMargin=36,
            bottomMargin=36
        )
        
        # Generate visualizations if data is provided
        if price_data is not None:
            self.generate_visualizations(price_data)
        
        # Report content - build as a single list of elements
        story = []
        
        # 1. Title page
        story.extend(self._create_title_page())
        story.append(PageBreak())
        
        # 2. Data overview
        story.extend(self._create_data_overview())
        story.append(PageBreak())
        
        # 3. Data quality dashboard
        story.extend(self._create_data_quality_dashboard())
        story.append(PageBreak())
        
        # 4. Statistical summary with visualizations
        story.extend(self._create_statistical_summary(price_data))
        
        # 5. Generate PDF
        doc.build(story)
        
        print(f"Generated report: {report_filename}")
        return report_filename
    
    def generate_visualizations(self, price_data):
        """Generate all necessary visualizations"""
        # Create summary dashboard
        self._create_summary_dashboard(price_data)
        
        # Create sample ETF visualizations
        sample_etfs = price_data['Ticker'].unique()[:5]  # First 5 ETFs
        for ticker in sample_etfs:
            self._create_etf_visualization(price_data, ticker)
    
    def _create_summary_dashboard(self, price_data):
        """Create data quality dashboard visualization"""
        plt.figure(figsize=(16, 12))
        
        # 1. Price Distribution
        plt.subplot(2, 2, 1)
        sns.histplot(price_data['close'], kde=True, bins=50)
        plt.title('Closing Price Distribution')
        plt.xlabel('Price')
        plt.ylabel('Frequency')
        
        # 2. Daily Returns
        plt.subplot(2, 2, 2)
        returns = price_data.groupby('Ticker')['close'].pct_change().dropna()
        sns.histplot(returns, kde=True, bins=100)
        plt.title('Daily Returns Distribution')
        plt.xlabel('Daily Return')
        plt.ylabel('Frequency')
        plt.xlim(-0.1, 0.1)
        
        # 3. Volume Analysis
        plt.subplot(2, 2, 3)
        volume_data = price_data[price_data['volume'] > 0]  # Filter zero volume
        volume_data['log_volume'] = np.log10(volume_data['volume'])
        sns.boxplot(y=volume_data['log_volume'])
        plt.title('Log Trading Volume Distribution')
        plt.ylabel('Log10(Volume)')
        
        # 4. Correlation Heatmap
        plt.subplot(2, 2, 4)
        corr_data = price_data[['open', 'high', 'low', 'close', 'volume']].corr()
        sns.heatmap(corr_data, annot=True, cmap='coolwarm', fmt=".2f")
        plt.title('Price-Volume Correlation')
        
        plt.tight_layout()
        dashboard_path = os.path.join(self.validation_dir, "data_quality_dashboard.png")
        plt.savefig(dashboard_path, dpi=150)
        plt.close()
    
    def _create_etf_visualization(self, price_data, ticker):
        """Create visualization for a single ETF"""
        etf_data = price_data[price_data['Ticker'] == ticker]
        if etf_data.empty:
            return
        
        plt.figure(figsize=(14, 10))
        
        # Price Trend
        plt.subplot(2, 2, 1)
        plt.plot(etf_data['date'], etf_data['close'])
        plt.title(f'{ticker} Price Trend')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.grid(True)
        
        # Daily Returns
        plt.subplot(2, 2, 2)
        returns = etf_data['close'].pct_change().dropna()
        sns.histplot(returns, kde=True, bins=50)
        plt.title(f'{ticker} Daily Returns')
        plt.xlabel('Daily Return')
        plt.ylabel('Frequency')
        
        # OHLC Boxplot
        plt.subplot(2, 2, 3)
        ohlc_data = etf_data[['open', 'high', 'low', 'close']]
        sns.boxplot(data=ohlc_data)
        plt.title(f'{ticker} Price Distribution')
        plt.ylabel('Price')
        
        # Volume Analysis
        plt.subplot(2, 2, 4)
        plt.bar(etf_data['date'], etf_data['volume'])
        plt.title(f'{ticker} Trading Volume')
        plt.xlabel('Date')
        plt.ylabel('Volume')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        viz_path = os.path.join(self.visualizations_dir, f"{ticker}_analysis.png")
        plt.savefig(viz_path, dpi=150)
        plt.close()
    
    def _create_title_page(self):
        """Create report title page"""
        elements = []
        
        # Title
        title = Paragraph("ETF Data Quality Report", self.title_style)
        elements.append(Spacer(1, 2 * inch))
        elements.append(title)
        
        # Subtitle
        subtitle = Paragraph("Comprehensive Analysis of Historical Price Data", 
                            ParagraphStyle('Subtitle', 
                                           parent=self.styles['Heading2'],
                                           fontSize=14,
                                           alignment=1,
                                           spaceBefore=20))
        elements.append(subtitle)
        
        # Company information
        company_info = [
            ["Generated by:", "Rui Xu"],
            ["Date:", datetime.now().strftime("%Y-%m-%d")],
            ["Data Coverage:", "1-Year Historical Prices"]
        ]
        
        company_table = Table(company_info, colWidths=[1.5*inch, 3*inch])
        company_table.setStyle(TableStyle([
            ('FONT', (0, 0), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 12),
            ('ALIGN', (0, 0), (0, -1), 'RIGHT'),
            ('ALIGN', (1, 0), (1, -1), 'LEFT'),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('TEXTCOLOR', (0, 0), (-1, -1), colors.HexColor("#2c3e50")),
        ]))
        
        elements.append(Spacer(1, 1.5 * inch))
        elements.append(company_table)
        
        return elements
    
    def _create_data_overview(self):
        """Create data overview page"""
        elements = []
        
        # Title
        section_title = Paragraph("Data Overview", self.section_style)
        elements.append(section_title)
        elements.append(Spacer(1, 0.2 * inch))
        
        # Overview data (simulated)
        overview_data = [
            ["Total ETFs Analyzed", "287"],
            ["Data Coverage Period", "2023-06-01 to 2024-06-01"],
            ["Total Records", "72,845"],
            ["Average Records per ETF", "254"],
            ["ETFs with Missing Data", "23 (8.0%)"],
            ["ETFs with Anomalies", "67 (23.3%)"]
        ]
        
        # Create table
        overview_table = Table(
            [[Paragraph(item[0], self.normal_style), item[1]] for item in overview_data],
            colWidths=[3*inch, 2*inch]
        )
        
        # Table style
        overview_style = TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#3498db")),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor("#ecf0f1")),
            ('GRID', (0, 0), (-1, -1), 1, colors.HexColor("#bdc3c7")),
            ('ALIGN', (1, 0), (1, -1), 'CENTER'),
        ])
        
        overview_table.setStyle(overview_style)
        elements.append(overview_table)
        elements.append(Spacer(1, 0.3 * inch))
        
        # Key metrics
        metrics_title = Paragraph("Key Quality Metrics", self.section_style)
        elements.append(metrics_title)
        elements.append(Spacer(1, 0.1 * inch))
        
        metrics_data = [
            ["Metric", "Value", "Status"],
            ["Data Completeness", "98.7%", "Excellent"],
            ["Anomaly Rate", "0.45%", "Acceptable"],
            ["Zero Volume Days", "0.12%", "Good"],
        ]
        
        metrics_table = Table(metrics_data, colWidths=[2*inch, 1.5*inch, 1.5*inch])
        
        # Add status colors
        status_colors = {
            "Excellent": colors.HexColor("#2ecc71"),
            "Good": colors.HexColor("#3498db"),
            "Acceptable": colors.HexColor("#f39c12"),
            "Poor": colors.HexColor("#e74c3c")
        }
        
        metrics_style = TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#2c3e50")),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor("#ecf0f1")),
            ('GRID', (0, 0), (-1, -1), 1, colors.HexColor("#bdc3c7")),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ])
        
        # Color status column
        for i in range(1, len(metrics_data)):
            status = metrics_data[i][2]
            metrics_style.add('BACKGROUND', (2, i), (2, i), status_colors.get(status, colors.white))
        
        metrics_table.setStyle(metrics_style)
        elements.append(metrics_table)
        
        return elements
    
    def _create_data_quality_dashboard(self):
        """Create data quality dashboard page"""
        elements = []
        
        # Title
        section_title = Paragraph("Data Quality Dashboard", self.section_style)
        elements.append(section_title)
        elements.append(Spacer(1, 0.2 * inch))
        
        # Dashboard image
        dashboard_img = os.path.join(self.validation_dir, "data_quality_dashboard.png")
        
        if os.path.exists(dashboard_img):
            dashboard = Image(dashboard_img, width=9*inch, height=5*inch)
            elements.append(dashboard)
        else:
            elements.append(Paragraph("Dashboard image not available", self.normal_style))
        
        elements.append(Spacer(1, 0.2 * inch))
        
        # Dashboard interpretation
        interpretation = [
            "The dashboard provides an overview of ETF dataset quality:",
            "- <b>Price Distribution</b>: Shows the range of closing prices across all ETFs",
            "- <b>Daily Returns</b>: Distribution of daily price changes",
            "- <b>Trading Volume</b>: Distribution of log-transformed volume data",
            "- <b>Correlation</b>: Relationships between price and volume metrics"
        ]
        
        for item in interpretation:
            elements.append(Paragraph(item, self.normal_style))
            elements.append(Spacer(1, 0.05 * inch))
        
        return elements
    
    def _create_statistical_summary(self, price_data=None):
        """Create statistical summary page with visualizations"""
        elements = []
        
        # Title
        section_title = Paragraph("Statistical Summary", self.section_style)
        elements.append(section_title)
        elements.append(Spacer(1, 0.2 * inch))
        
        # Key statistical indicators
        stats_title = Paragraph("Key Statistical Indicators", self.section_style)
        elements.append(stats_title)
        elements.append(Spacer(1, 0.1 * inch))
        
        # Calculate statistics if data is available
        if price_data is not None:
            # Calculate statistics
            stats = {
                "Close Price": {
                    "Mean": price_data['close'].mean(),
                    "Median": price_data['close'].median(),
                    "Std Dev": price_data['close'].std(),
                    "Min": price_data['close'].min(),
                    "Max": price_data['close'].max(),
                },
                "Daily Volume": {
                    "Mean": price_data['volume'].mean(),
                    "Median": price_data['volume'].median(),
                    "Std Dev": price_data['volume'].std(),
                    "Min": price_data['volume'].min(),
                    "Max": price_data['volume'].max(),
                },
                "Daily Return": {
                    "Mean": price_data.groupby('Ticker')['close'].pct_change().mean(),
                    "Std Dev": price_data.groupby('Ticker')['close'].pct_change().std(),
                    "Min": price_data.groupby('Ticker')['close'].pct_change().min(),
                    "Max": price_data.groupby('Ticker')['close'].pct_change().max(),
                }
            }
            
            # Format statistics
            stats_data = [
                ["Statistic", "Close Price", "Daily Volume", "Daily Return"],
                ["Mean", f"{stats['Close Price']['Mean']:.2f}", 
                 f"{stats['Daily Volume']['Mean']:,.0f}", 
                 f"{stats['Daily Return']['Mean']:.2%}"],
                ["Median", f"{stats['Close Price']['Median']:.2f}", 
                 f"{stats['Daily Volume']['Median']:,.0f}", 
                 "N/A"],
                ["Std Dev", f"{stats['Close Price']['Std Dev']:.2f}", 
                 f"{stats['Daily Volume']['Std Dev']:,.0f}", 
                 f"{stats['Daily Return']['Std Dev']:.2%}"],
                ["Min", f"{stats['Close Price']['Min']:.2f}", 
                 f"{stats['Daily Volume']['Min']:,.0f}", 
                 f"{stats['Daily Return']['Min']:.2%}"],
                ["Max", f"{stats['Close Price']['Max']:.2f}", 
                 f"{stats['Daily Volume']['Max']:,.0f}", 
                 f"{stats['Daily Return']['Max']:.2%}"],
            ]
        else:
            # Use sample data if no price data available
            stats_data = [
                ["Statistic", "Close Price", "Daily Volume", "Daily Return"],
                ["Mean", "152.34", "2,450,000", "0.08%"],
                ["Median", "146.72", "1,890,000", "N/A"],
                ["Std Dev", "18.67", "1,230,000", "1.24%"],
                ["Min", "102.45", "150,000", "-8.76%"],
                ["Max", "210.89", "15,780,000", "9.45%"],
            ]
        
        stats_table = Table(stats_data)
        stats_table.setStyle(self.table_style)
        elements.append(stats_table)
        elements.append(Spacer(1, 0.3 * inch))
        
        # Add ETF visualizations if available
        if os.path.exists(self.visualizations_dir):
            viz_files = [f for f in os.listdir(self.visualizations_dir) 
                         if f.endswith('_analysis.png')]
            sample_viz = viz_files[:3]  # Show up to 3 sample visualizations
            
            if sample_viz:
                viz_title = Paragraph("Sample ETF Visualizations", self.section_style)
                elements.append(viz_title)
                elements.append(Spacer(1, 0.1 * inch))
                
                for i, viz_file in enumerate(sample_viz):
                    # Get ETF ticker from filename
                    ticker = viz_file.split('_')[0]
                    
                    # Add ETF title
                    etf_title = Paragraph(f"ETF: {ticker}", self.section_style)
                    elements.append(etf_title)
                    
                    # Add visualization
                    img_path = os.path.join(self.visualizations_dir, viz_file)
                    if os.path.exists(img_path):
                        # Scale image to fit page
                        img = Image(img_path, width=7*inch, height=5*inch)
                        elements.append(img)
                    
                    # Add space between visualizations
                    if i < len(sample_viz) - 1:
                        elements.append(Spacer(1, 0.2 * inch))
        
        return elements

# Usage example
if __name__ == "__main__":
    # Sample price data (in a real scenario, load from database or CSV)
    dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='B')
    tickers = ['SPY', 'QQQ', 'DIA', 'IWM', 'GLD']
    
    price_data = pd.DataFrame({
        'date': np.tile(dates, len(tickers)),
        'Ticker': np.repeat(tickers, len(dates)),
        'open': np.random.uniform(100, 200, len(dates)*len(tickers)),
        'high': np.random.uniform(100, 210, len(dates)*len(tickers)),
        'low': np.random.uniform(90, 190, len(dates)*len(tickers)),
        'close': np.random.uniform(100, 200, len(dates)*len(tickers)),
        'volume': np.random.randint(1000000, 10000000, len(dates)*len(tickers))
    })
    
    # Create report generator
    report_generator = ETFReportGenerator(validation_dir="validation_results")
    
    # Generate report with visualizations
    report_path = report_generator.generate_full_report(price_data=price_data)
    
    print(f"ETF Data Quality Report with visualizations generated at: {report_path}")

locator: <matplotlib.ticker.AutoLocator object at 0x16a430200>


locator: <matplotlib.ticker.AutoLocator object at 0x16a430200>
locator: <matplotlib.ticker.AutoLocator object at 0x16a430200>
locator: <matplotlib.ticker.AutoLocator object at 0x16a430200>


locator: <matplotlib.ticker.AutoLocator object at 0x16a430200>


locator: <matplotlib.ticker.AutoLocator object at 0x16a430200>


STREAM b'IHDR' 16 13


...

STREAM b'IDAT' 131 65536


STREAM b'IDAT' 131 65536
STREAM b'IDAT' 131 65536
STREAM b'IDAT' 131 65536


STREAM b'IDAT' 131 65536


STREAM b'IDAT' 131 65536


Generated report: validation_results/reports/ETF_Data_Quality_Report_20250623_004108.pdf
ETF Data Quality Report with visualizations generated at: validation_results/reports/ETF_Data_Quality_Report_20250623_004108.pdf
